In [4]:
import os
import cv2
import glob
import numpy as np
import pandas as pd
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [6]:
# resize image
def show_image(array):
    display(image.array_to_img(array))

def batch_resize_image():
    image_list = glob.glob('../data/raw_image/*')
    for image_path in image_list:
        img = image.load_img(image_path, target_size=(image_size, image_size))
        img.save('../data/resized_image/{}'.format(os.path.basename(image_path)))

image_size = 256
batch_resize_image()

OSError: cannot identify image file '../data/raw_image/IMG_4477.jpg'

In [4]:
# y_coord
import json

def get_coords(x):
    coord_json = json.loads(x)
    result_list = []
    for geometry in coord_json['Corner of Building']:
        result_list.append([geometry['geometry']['x'], geometry['geometry']['y']])
    return result_list

def calc_distance(p1, p2):
    return (p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2

def find_nearest_coord(x, point):
    result_list = []
    for p in x:
        d = calc_distance(point, p)
        result_list.append(d)
    nearest_coord = x[np.argmin(result_list)]
    return x[np.argmin(result_list)]

def reformat_coord(df, image_size):
    tmp_list = []
    for point in df.values:
        [tmp_list.append(coord / image_size) for coord in point]
    return tmp_list

# 手动标注端点坐标的图片大小为512
# last_index = 511
# coord_df = pd.read_csv('../data/coner_coord/export-2019-02-08T05_15_26.966Z.csv')[['External ID', 'Label']]
# coord_df['coords'] = coord_df['Label'].apply(lambda x: get_coords(x))
# coord_df['ul'] = coord_df['coords'].apply(lambda x: find_nearest_coord(x, [0, 0]))
# coord_df['ur'] = coord_df['coords'].apply(lambda x: find_nearest_coord(x, [last_index, 0]))
# coord_df['lr'] = coord_df['coords'].apply(lambda x: find_nearest_coord(x, [last_index, last_index]))
# coord_df['ll'] = coord_df['coords'].apply(lambda x: find_nearest_coord(x, [0, last_index]))
# coord_df[['ul', 'ur', 'lr', 'll']].to_csv('../data/coner_coord/y_coords.csv', index=False)

In [37]:
# get_mask
def batch_get_mask(coord_df):
    dimension = 1
    result_list = []
    for i in range(len(coord_df)):
        image_path = '../data/resized_image/{}'.format(coord_df.iloc[i]['External ID'])
        mask_path = '../data/resized_mask/{}'.format(coord_df.iloc[i]['External ID'])
        points = coord_df.iloc[i][['ul', 'ur', 'lr', 'll']].values
        
        if dimension == 1:
            single_image = np.zeros((corner_size, corner_size, dimension))
        else:
            single_image = image.load_img(image_path)
            single_image = image.img_to_array(single_image)
        cv2.fillConvexPoly(single_image, np.array([point for point in points]), (255) if dimension == 1 else (255, 255, 255))
        
        new_size = 256
        img = image.array_to_img(single_image)
        img = img.resize((new_size, new_size))
        img.save(mask_path)
        # display(img)

batch_get_mask(coord_df)

In [38]:
# generate more image and mask
image_list = glob.glob('../data/resized_image/*')
mask_list = glob.glob('../data/resized_mask/*')
image_result = []
mask_result = []
for image_path, mask_path in zip(image_list, mask_list):
    image_result.append(image.img_to_array(image.load_img(image_path)))
    mask_result.append(image.img_to_array(image.load_img(mask_path, grayscale=True)))
image_result = np.array(image_result)
mask_result = np.array(mask_result)

In [39]:
# generate more image and mask
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

image_args = dict(featurewise_center = False, 
                  samplewise_center = False,
                  rotation_range = 5, 
                  width_shift_range = 0.01, 
                  height_shift_range = 0.01, 
                  shear_range = 0.01,
                  zoom_range = [0.9, 1.1],  
                  horizontal_flip = False, 
                  vertical_flip = False,
                  fill_mode = 'nearest',
                  data_format = 'channels_last',
                  preprocessing_function = preprocess_input
                 )
mask_args = image_args.copy()
mask_args['preprocessing_function'] = lambda x: x / 255.0
image_datagen = ImageDataGenerator(**image_args)
mask_datagen = ImageDataGenerator(**mask_args)

seed = 2
batch_size = 20
image_flow_args = dict(x = image_result,
                       batch_size = batch_size, 
                       shuffle = True, 
                       seed = seed)
mask_flow_args = dict(x = mask_result,
                      batch_size = batch_size, 
                      shuffle = True, 
                      seed = seed)

loop_count = 0
file_index = 0
gen_image_result = []
gen_mask_result = []
for new_images, new_masks in zip(image_datagen.flow(**image_flow_args), mask_datagen.flow(**mask_flow_args)):
    gen_image_result.extend(new_images)
    gen_mask_result.extend(new_masks)

    for new_image, new_mask in zip(new_images, new_masks):
        img = image.array_to_img(new_image)
        mask = image.array_to_img(new_mask)
        img.save('../data/gen_image/{}.jpg'.format(str(file_index).zfill(4)))
        mask.save('../data/gen_mask/{}.jpg'.format(str(file_index).zfill(4)))
        file_index += 1
        
    if loop_count >= 40:
        break
    loop_count += 1
    
gen_image_result = np.array(gen_image_result)
gen_mask_result = np.array(gen_mask_result)
print(gen_image_result.max(), gen_mask_result.max())
print(gen_image_result.shape, gen_mask_result.shape)
np.save('../data/cnn_data/x.npy', gen_image_result)
np.save('../data/cnn_data/y_256.npy', gen_mask_result)

151.061 1.0
(640, 256, 256, 3) (640, 256, 256, 1)


In [24]:
# # 把y做的更小
# target_size = 64
# resized_y_list = []
# for y in data_y:
#     resized_y_list.append(image.img_to_array(image.array_to_img(y).resize((target_size, target_size))))
# resized_y_array = np.array(resized_y_list) / 255.0
# print(resized_y_array.shape, resized_y_array.max())
# np.save('../data/cnn_data/y_{}.npy'.format(target_size), resized_y_array)

In [11]:
# # resized_image的四角坐标
# def get_corner_coords_from_mask(target_array):
#     dst = cv2.dilate(cv2.cornerHarris(target_array,2,3,0.04),None)
#     item_index = np.where(dst>0.1*dst.max())
#     item_index_array = np.concatenate([np.expand_dims(item_index[1], axis=-1), np.expand_dims(item_index[0], axis=-1)], axis=1)
#     ul = item_index_array[np.argmin([calc_distance(x, [0, 0]) for x in item_index_array])]
#     ur = item_index_array[np.argmin([calc_distance(x, [0, last_index]) for x in item_index_array])]
#     lr = item_index_array[np.argmin([calc_distance(x, [last_index, last_index]) for x in item_index_array])]
#     ll = item_index_array[np.argmin([calc_distance(x, [last_index, 0]) for x in item_index_array])]
#     return [ul[0], ul[1], ur[0], ur[1], lr[0], lr[1], ll[0], ll[1]]
    
# last_index = 255
# data_y = np.load('../data/cnn_data/y_256.npy')
# resized_image_coords = np.array([get_corner_coords_from_mask(target_array) for target_array in data_y]) / 255.0
# np.save('../data/cnn_data/y_{}_coords.npy'.format(256), resized_image_coords)

for i in range(3):
    y_true_mask = data_y[i]
#     show_image(y_true_mask)
    
    y_pred = resized_image_coords[i]
    points = np.array([[int(y_pred[0]*255.0),int(y_pred[1]*255.0)], 
                       [int(y_pred[2]*255.0),int(y_pred[3]*255.0)], 
                       [int(y_pred[4]*255.0),int(y_pred[5]*255.0)], 
                       [int(y_pred[6]*255.0),int(y_pred[7]*255.0)]])
    y_pred_mask = np.zeros((256, 256, 1)).astype(np.float32)
    cv2.fillConvexPoly(y_pred_mask, points, (1))
#     show_image(y_pred_mask)

    print(type(y_true_mask), type(y_pred_mask))
    print(y_true_mask.shape, y_pred_mask.shape)
    print((y_true_mask * y_pred_mask).shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(256, 256, 1) (256, 256, 1)
(256, 256, 1)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(256, 256, 1) (256, 256, 1)
(256, 256, 1)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(256, 256, 1) (256, 256, 1)
(256, 256, 1)
